Importación de librerías base para el problema 

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px
import os


os.getcwd()


'c:\\Users\\shado\\OneDrive\\Escritorio\\Latam\\LATAM'

# 2.- Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv :


#### Se inició por la segunda parte antes que el primer punto, ya que también analizare las variables agregadas externas al dataframe, por esta razón el punto 1 y 3 de análisis se estarán más abajo.

#### Lectura de datos principal y creacion de copia auxiliar de de la data original, también se busco información extra sobre países de los destinos de vuelos y los días feriados de ese año para el origen de los vuelos

#### Lectura y creación de variables en el data frame 

In [2]:
datos = pd.read_csv("dataset_SCL.csv", sep=",")
datos_original = datos.copy()
paises =  pd.read_excel("Prefijo Paises.xlsx", sheet_name= 'Hoja2')
feriados =  pd.read_excel("Feriados.xlsx")
datos.columns


C:\Users\shado\AppData\Local\Temp\ipykernel_32352\200450372.py:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv("dataset_SCL.csv", sep=",")


Index(['Fecha-I', 'Vlo-I', 'Ori-I', 'Des-I', 'Emp-I', 'Fecha-O', 'Vlo-O',
       'Ori-O', 'Des-O', 'Emp-O', 'DIA', 'MES', 'AÑO', 'DIANOM', 'TIPOVUELO',
       'OPERA', 'SIGLAORI', 'SIGLADES'],
      dtype='object')

#### Verificación de tipos de datos en el data frame

In [3]:
datos.dtypes

Fecha-I      object
Vlo-I        object
Ori-I        object
Des-I        object
Emp-I        object
Fecha-O      object
Vlo-O        object
Ori-O        object
Des-O        object
Emp-O        object
DIA           int64
MES           int64
AÑO           int64
DIANOM       object
TIPOVUELO    object
OPERA        object
SIGLAORI     object
SIGLADES     object
dtype: object

transformanción de Fechas a formato tiempo

In [4]:
datos["Fecha-I"] = pd.to_datetime(datos["Fecha-I"])
datos["Fecha-O"] = pd.to_datetime(datos["Fecha-O"])
datos.dtypes

Fecha-I      datetime64[ns]
Vlo-I                object
Ori-I                object
Des-I                object
Emp-I                object
Fecha-O      datetime64[ns]
Vlo-O                object
Ori-O                object
Des-O                object
Emp-O                object
DIA                   int64
MES                   int64
AÑO                   int64
DIANOM               object
TIPOVUELO            object
OPERA                object
SIGLAORI             object
SIGLADES             object
dtype: object

creación de variables dif_min, hora_dia, hora, minutos

In [5]:
datos['dif_min'] = (datos["Fecha-O"]- datos["Fecha-I"])
datos['hora_dia'] = datos['Fecha-I'].dt.time
datos['hora'] = datos['Fecha-I'].dt.hour
datos['minutos'] = datos['Fecha-I'].dt.minute

datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,2017,Martes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,2017,Miercoles,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,2017,Jueves,I,American Airlines,Santiago,Miami,-1 days +23:58:00,23:30:00,23,30


cambia la diferencia de las fechas de programación y salida de los vuelos a minutos.

In [6]:
datos["dif_min"] =  datos["dif_min"]/np.timedelta64(1,'m')

Homologación de variable Mes a nombre

In [7]:
conditions = [
    (datos['MES'] == 1),
    (datos['MES'] == 2),
    (datos['MES'] == 3),
    (datos['MES'] == 4),
    (datos['MES'] == 5),
    (datos['MES'] == 6),
    (datos['MES'] == 7),
    (datos['MES'] == 8),
    (datos['MES'] == 9),
    (datos['MES'] == 10),
    (datos['MES'] == 11),
    (datos['MES'] == 12)]
values = [
    'Enero',
    'Febrero',
    'Marzo',
    'Abril',
    'Mayo',
    'Junio',
    'Julio',
    'Agosto',
    'Septiembre',
    'Octubre',
    'Noviembre',
    'Diciembre']
datos['Mes_Nombre'] = np.select(conditions, values)
datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos,Mes_Nombre
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,Domingo,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,Lunes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,Martes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,Miercoles,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,Jueves,I,American Airlines,Santiago,Miami,-2.0,23:30:00,23,30,Enero


Verificacion que el valores este correcto

In [8]:
datos[["dif_min","Fecha-O","Fecha-I"]]

,dif_min,Fecha-O,Fecha-I
0,3.0,2017-01-01 23:33:00,2017-01-01 23:30:00
1,9.0,2017-01-02 23:39:00,2017-01-02 23:30:00
2,9.0,2017-01-03 23:39:00,2017-01-03 23:30:00
3,3.0,2017-01-04 23:33:00,2017-01-04 23:30:00
4,-2.0,2017-01-05 23:28:00,2017-01-05 23:30:00
...,...,...,...
68201,46.0,2017-12-22 15:41:00,2017-12-22 14:55:00
68202,16.0,2017-12-25 15:11:00,2017-12-25 14:55:00
68203,40.0,2017-12-27 15:35:00,2017-12-27 14:55:00
68204,13.0,2017-12-29 15:08:00,2017-12-29 14:55:00


Creacion de variable temporada_alta y atraso_15


In [9]:
datos['temporada_alta'] = np.where(  ((datos['Fecha-I']>='2017-12-15') & (datos['Fecha-I']<='2018-03-03')) 
                                   | ((datos['Fecha-I']>='2017-07-15') & (datos['Fecha-I']<='2017-07-31'))
                                   | ((datos['Fecha-I']>='2017-01-01') & (datos['Fecha-I']<='2017-03-03'))
                                   | ((datos['Fecha-I']>='2017-09-11') & (datos['Fecha-I']<='2017-09-30'))
                                   ,1,0)
datos['atraso_15'] = np.where(datos['dif_min']>15,1,0)




#### creacion de variable periodo_dia

In [10]:
conditions = [
    (datos['hora'] >= 5) & (datos['hora'] < 12),
    (datos['hora'] >= 12) & (datos['hora'] < 19),
    (datos['hora'] >= 19) | ((datos['hora'] >= 0) & (datos['hora'] < 5))]
values = ['mañana', 'tarde', 'noche']
datos['periodo_dia'] = np.select(conditions, values)


##### Creación Variable Destino, la cual agrupa Destino de operación cuales tengan menos de 10 vuelos en categoria Otros

In [11]:
Destino_pre = datos.groupby(['Des-O']
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        , dropna=False, as_index= False).agg(
    Cantidad_Vuelos = ('atraso_15','count'))


Destino_menores = Destino_pre[Destino_pre['Cantidad_Vuelos']<=10]

flex = Destino_menores['Des-O'].to_list() #agrega lista de destinos con menos de 10 vuelos
datos['Destino'] = np.where(datos['Des-O'].isin(flex), 'Otro',datos['Des-O']) #Creación de variable Destino donde se agrupan los DES-O con menos de 10 vuelos en categoria Otros

#### Creación de variable País

In [12]:

paises['Des-O_2'] = paises['Código prefijo']
paises['Largo'] = paises['Des-O_2'].str.len()

paises_1 = paises[paises['Largo'] == 1]
paises_2 = paises[paises['Largo'] == 2]

def subs(x):
    return x[:2]
datos['Des-O_2'] = datos['Des-O'].apply(subs)
datos['Des-O_2']
paises_2 = paises_2.drop(columns=['Largo','Código prefijo'], axis=1)
paises_1 = paises_1.drop(columns=['Largo','Código prefijo'], axis=1)



##### se crean dos set de datos uno con los paises de prefijo largo 1 y otro con los paises de prefijo largo 2, se trabajan por separado y luego se juntan solo en el data frame de datos.

In [13]:
datos_test = datos.merge(paises_2, how='left', left_on='Des-O_2', right_on = 'Des-O_2')
datos_1 = datos_test[datos_test['País'].isnull()]
datos_2 = datos_test[datos_test['País'].notnull()]

datos_1 =  datos_1.drop(columns=[ 'País', ], axis=1)

def subs(x):
    return x[:1]
datos_1['Des-O_2'] = datos_1['Des-O'].apply(subs)
datos_1['Des-O_2']

datos_1 = datos_1.merge(paises_1, how='left', left_on='Des-O_2', right_on = 'Des-O_2')
datos_test = pd.concat([datos_1, datos_2], axis = 0)

datos_test =  datos_test.drop(columns=['LARGO_x','LARGO_y', 'LARGO'], axis=1)

datos_test =  datos_test.drop(columns=[ 'Des-O_2', ], axis=1)
datos_test


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,dif_min,hora_dia,hora,minutos,Mes_Nombre,temporada_alta,atraso_15,periodo_dia,Destino,País
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,3.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,9.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,9.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,3.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,-2.0,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,46.0,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,16.0,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,40.0,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,13.0,14:55:00,14,55,Diciembre,1,0,tarde,SPJC,Perú


#### Creación de una variable con los dias Feriados en el data frame.

In [14]:
datos_test['Fecha-I'] = datos_test['Fecha-I'].dt.normalize()
datos_test = datos_test.merge(feriados, how='left', left_on='Fecha-I', right_on = 'Fecha-I')
datos_test['Feriado'] = np.where(datos_test['Feriado'].isnull(),'No Feriado','Feriado')
datos_test


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,hora_dia,hora,minutos,Mes_Nombre,temporada_alta,atraso_15,periodo_dia,Destino,País,Feriado
0,2017-01-01,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,Feriado
1,2017-01-02,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,No Feriado
2,2017-01-03,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,No Feriado
3,2017-01-04,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,No Feriado
4,2017-01-05,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,23:30:00,23,30,Enero,1,0,noche,KMIA,Estados contiguos de los Estados Unidos,No Feriado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú,No Feriado
68202,2017-12-25,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú,Feriado
68203,2017-12-27,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,14:55:00,14,55,Diciembre,1,1,tarde,SPJC,Perú,No Feriado
68204,2017-12-29,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,14:55:00,14,55,Diciembre,1,0,tarde,SPJC,Perú,No Feriado


exportacion de archivo csv con las variables creadas desde el dataframe

In [15]:
synthetic_features = datos_test[[
            'temporada_alta'
            ,'dif_min'
            ,'atraso_15'
            ,'periodo_dia'
            ,'Destino'
            ,'País'
            ,'Feriado'
            ]]

synthetic_features.to_csv("synthetic_features.csv"
                          , sep="|"
                          , header=True
                          , index=False)

# 1.- ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?

In [16]:
fig = px.histogram(datos_test, x="dif_min",
                   title='Histograma minutos de diferencia',
                   labels={'dif_min':'total bill'}, 
                   opacity=0.7,
                   color_discrete_sequence=['#FF2800'] 
                   )
fig.update_traces(marker_line_width = 1
                  ,marker_line_color = 'black')


fig.show()



#### Vamos a verificar distribución de la variable Minutos de diferencia entre la programación del vuelo y la salida de este. observamos que la data se encuentra cargada a la izquierda con valores negativos, lo que nos dice que existen algunos vuelos que salen antes de lo programado. Para nuestro ejemplo nos centraremos en analizar los datos que presentan atrasos ya que es la variable que intentaremos predecir posteriormente.

In [17]:
atrasos = datos[datos['atraso_15']==1] #filtro para ver histograma solo de vuelos con atrasos


fig = px.histogram(atrasos
                   , x="dif_min"
                   ,title='Histograma minutos de diferencia'
                   ,labels={'dif_min':'total bill'}
                   ,opacity=0.8
                   ,color_discrete_sequence=['#FF2800'] )

fig.update_traces(marker_line_width = 1
                  ,marker_line_color = 'black')
fig.show()



#### Aquí se observa una figura muy parecida a los que es un gráfico de una variable exponencial, verificaremos como se porta una variable exponencial teórica con parámetro lambda usando el estimador de máxima verosimilitud de la muestra.

# 3.-   ¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo?
#       ¿Qué variables esperarías que más influyeran en predecir atrasos?


### En la siguiente sección se entregaran diversas estadísticas descriptivas y análisis de datos para ir respondiendo esta consulta.

#### Calculo de tasa de atrasos de la base total

In [18]:
Tasa_atrasos = datos_test.groupby(['atraso_15']
                         # 'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        , dropna=False,as_index=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'))
Tasa_atrasos['Tasa de atrasos'] = (Tasa_atrasos['Cantidad_Vuelos']/68203)*100


def f(row):
    if row == 0:
        val = 'No Atraso'
    else:
        val = 'Atraso'
    return val
Tasa_atrasos['atraso_15'] = Tasa_atrasos['atraso_15'].apply(f)
print(Tasa_atrasos)




   atraso_15  Cantidad_Vuelos  Tasa de atrasos
0  No Atraso            55592        81.509611
1     Atraso            12614        18.494788


#### Grafico tasa de atrasos en la data completa


In [19]:

fig = px.bar(Tasa_atrasos
             , y ='atraso_15'
             , x ='Tasa de atrasos'
             , color_continuous_scale= 'Bluered'
             , color='Cantidad_Vuelos'
             , orientation= 'h'
             , title = 'Tasa de atrasos'
             , text_auto = True
             )
fig.show()

### Se observa que el 18.49% de los vuelos generales presentan atrasos

### Calculo de tasa de atrasos por Destino, se toma en cuenta para esta tasa destinos mayores o iguales a 10 vuelos para la variable Destino de Operacion Des-O  

In [20]:
Destino_pre = datos.groupby(['Des-O']
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        , dropna=False, as_index= False).agg(
    Cantidad_Vuelos = ('atraso_15','count'))


Destino_menores = Destino_pre[Destino_pre['Cantidad_Vuelos']<=10]

flex = Destino_menores['Des-O'].to_list() #agrega lista de destinos con menos de 10 vuelos
datos['Destino'] = np.where(datos['Des-O'].isin(flex), 'Otro',datos['Des-O']) #Creación de variable Destino donde se agrupan los DES-O con menos de 10 vuelos en categoria Otros


### Calculo de tasa de atrasos por Destino

In [21]:
Destino = datos_test.groupby(['Destino']
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        ,as_index=False, dropna=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_Atrasos = ('atraso_15','sum'))
                        
Destino['Tasa de atrasos'] = Destino['Cantidad_Atrasos']/Destino['Cantidad_Vuelos']*100 #Calculo tasa de atrasos
                       
Destino[Destino['Cantidad_Vuelos']>10].sort_values('Tasa de atrasos',ascending=False).head(10)
Destino_mayores = Destino[Destino['Cantidad_Vuelos']>10].sort_values('Tasa de atrasos',ascending=False)
Destino.sort_values('Tasa de atrasos',ascending=False)


,Destino,Cantidad_Vuelos,Cantidad_Atrasos,Tasa de atrasos
53,YSSY,194,113,58.247423
52,YMML,39,21,53.846154
17,Otro,25,10,40.000000
18,SAAR,136,54,39.705882
16,NZAA,361,135,37.396122
12,MDPC,54,18,33.333333
0,CYYZ,283,89,31.448763
20,SACO,1023,308,30.107527
46,SGAS,232,66,28.448276
27,SBGL,1435,405,28.222997


### Se observa que de los Destinos de operación posibles el que presenta mayor tasa de atrasos es el de YSSY  y YMML ambos en Australia

### Grafíco de la tasa de atrasos por Destino

In [22]:
fig = px.bar(Destino_mayores.head(10).sort_values('Tasa de atrasos',ascending=True)
             , y ='Destino'
             , x ='Tasa de atrasos'
             , color_continuous_scale= 'Bluered'
             , color='Cantidad_Atrasos'
             , orientation= 'h'
             , title = 'Ranking (Top 10) de la Tasa de atrasos por destino segun cantidad de atrasos'
             , text_auto= True
             )
fig.show()

#### Gráfico de tasa de atrasos segun Destino (Top 10)

#### ¿es posible saber si Australia es el pais de destino con mayor tasa de atrasos?. Para responder esto usaremos los datos extraidos de  codificacion  Organización de Aviación Civil Internacional "OACI"

In [23]:
Pais = datos_test.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'País'
                        ], dropna=False, as_index=False).agg(
    Cantidad_vuelos = ('atraso_15','count'),
    Cantidad_atrasos = ('atraso_15','sum')
                                            )

Pais['Tasa de atrasos'] = (Pais['Cantidad_atrasos']/Pais['Cantidad_vuelos'])*100

Pais = Pais.sort_values('Tasa de atrasos',ascending=False)
Pais



,País,Cantidad_vuelos,Cantidad_atrasos,Tasa de atrasos
1,Australia,233,134,57.510730
13,Nueva Zelanda,361,135,37.396122
18,República Dominicana,54,18,33.333333
4,Canadá,283,89,31.448763
15,Paraguay,232,66,28.448276
0,Argentina,9237,2452,26.545415
6,Colombia,1604,415,25.872818
3,"Brasil (también SD, SN, SS y SW)",5171,1316,25.449623
16,Perú,5271,1214,23.031683
19,Uruguay,1072,221,20.615672


#### Calculamos la tasa de atraso por Pais de destino y podemos obsersvar que efevtivamente Australia es el pais que presenta la mayor tasa de atrasos en los viajes. 

In [24]:
fig = px.bar(Pais.head(15).sort_values('Tasa de atrasos',ascending=True)
            , y='País'
            , x='Tasa de atrasos'
            , color='Cantidad_atrasos'
            , text_auto= True
            )


fig.show()

#### Se muestran el top 10 de las tasas de atrasos por País.

### Gráfico de tasa de atrasos por país de Destino coloreado segun la suma de los atrasos. esto nos dice que la la tasa de atrasos mayor la presenta Australia como país, pero la mayor cantidad de atrasos la tiene Chile

#### Analísis de tasa de atrasos segun tipo de vuelo, Internacional o Nacional.

In [25]:
Tipo_Vuelo = datos_test.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'TIPOVUELO'
                        ], dropna=False, as_index=False).agg(
    Cantidad_vuelos = ('atraso_15','count'),
    Cantidad_atrasos = ('atraso_15','sum')
                                            )

Tipo_Vuelo['Tasa de atrasos'] = (Tipo_Vuelo['Cantidad_atrasos']/Tipo_Vuelo['Cantidad_vuelos'])*100




def f(row):
    if row == 'N':
        val = 'Nacional'
    else:
        val = 'Internacional'
    return val
    
Tipo_Vuelo['TIPOVUELO'] = Tipo_Vuelo['TIPOVUELO'].apply(f)
Tipo_Vuelo



,TIPOVUELO,Cantidad_vuelos,Cantidad_atrasos,Tasa de atrasos
0,Internacional,31240,7048,22.560819
1,Nacional,36966,5566,15.057079


In [26]:
fig = px.bar(Tipo_Vuelo.sort_values('Tasa de atrasos',ascending=True), y = 'TIPOVUELO', x = 'Tasa de atrasos', color = 'Cantidad_atrasos',text_auto=True)

fig.update_traces(textfont_size=12
                  , textangle=45
                  , textposition= 'inside'
                  ,marker_line_width=1
                  ,marker_line_color = 'black')

fig.show()

### Se observa que en los Vuelos Nacionales existe un 22.56% de tasa de atrasos vs un 15.05% de tasa de atrasos en los vuelos internacionales.

#### Calculo de Tasa de atrasos por Operador

In [27]:
Operador = datos.groupby([
                          #'Des-O'
                          'OPERA'
                          #'MES',
                          #'DIA',
                          #'temporada_alta'
                        ], dropna=False, as_index=False).agg(
    Cantidad_Vuelos = ('atraso_15','count'),
    Cantidad_atrasos = ('atraso_15','sum'))
Operador['Tasa de atrasos'] = (Operador['Cantidad_atrasos']/Operador['Cantidad_Vuelos'])*100
Operador = Operador.sort_values('Tasa de atrasos',ascending=False)
(Operador)


,OPERA,Cantidad_Vuelos,Cantidad_atrasos,Tasa de atrasos
19,Plus Ultra Lineas Aereas,49,30,61.224490
20,Qantas Airways,195,113,57.948718
2,Air Canada,565,258,45.663717
17,Latin American Wings,1674,681,40.681004
11,Gol Trans,806,287,35.607940
7,Avianca,1152,306,26.562500
0,Aerolineas Argentinas,1949,468,24.012314
6,Austral,74,15,20.270270
14,JetSmart SPA,1095,196,17.899543
12,Grupo LATAM,40892,7306,17.866575


#### En esta sección observamos que el Operador "Plus Ultra Lineas Aereas" presenta la mayor tasa de atrasos con 30 de sus 49 vuelos en total donde los sigue Qantas Airways con 113 de 195 y Air Canada con 258 de 565.

#### Grafico de tasa de atrasos por operador, coloreado por cantidad de atrasos 

In [28]:
fig = px.bar(Operador.head(15).sort_values('Tasa de atrasos',ascending=True)
            , y='OPERA' 
            ,  x='Tasa de atrasos'
            , color_continuous_scale= 'Bluered'
            , color='Cantidad_atrasos'
            , title= 'Tasa de atrasos por operador según cantidad de atrasos'
            , text_auto= True)
fig.update_traces( marker_line_color = 'black')
fig.show()

#### Observamos que los 3 Operadores que presentan mayor tasa de atrasos en sus vuelos son Plus Ultra Lineas Aereas, Qantas Airways y Air Canada 

#### Ahora observaremos como se ve reflejada la tasa de atrasos en función del tiempo y asignaciones, por ejemplo por mes, dia de la semana, periodos de día.

In [29]:
Mes = datos.groupby([
                          #'Des-O'
                          #'OPERA'
                          'Mes_Nombre'
                          #'DIA',
                          #'temporada_alta'
                        ], dropna=False, as_index=False).agg(
    Cantidad_vuelos = ('atraso_15','count'),
    Cantidad_atrasos = ('atraso_15','sum'))
Mes['Tasa de atrasos'] = (Mes['Cantidad_atrasos']/Mes['Cantidad_vuelos'])*100
Mes.sort_values('Tasa de atrasos',ascending=False)

,Mes_Nombre,Cantidad_vuelos,Cantidad_atrasos,Tasa de atrasos
5,Julio,5992,1758,29.339119
2,Diciembre,6356,1615,25.409062
10,Octubre,6032,1363,22.596154
9,Noviembre,6080,1199,19.720395
1,Agosto,5744,996,17.339833
11,Septiembre,5610,967,17.237077
6,Junio,4981,852,17.104999
3,Enero,6108,978,16.011788
8,Mayo,5240,827,15.782443
4,Febrero,5561,843,15.159144


#### Grafico de tasa de atrasos por mes coloreado por cantidad de atrasos 

In [30]:
fig = px.bar(Mes.sort_values('Tasa de atrasos'
                             ,ascending=True)
             , y='Mes_Nombre'
             , x='Tasa de atrasos'
             , color_continuous_scale= 'Bluered'
             , color='Cantidad_atrasos'
             , title= 'Tasa de atrasos por mes coloreado segun cantidad de atrasos'
             , text_auto= True)
            
fig.update_traces( marker_line_color = 'black')

fig.show()


### Podemos observar que el mes que presenta un mayor tasa de atrasos en sus vuelos es el mes de Julio con 29.33% de estos

### Estas cantidades en los atrasos del mes de Julio ¿seran observables a nivel diaro?, calcularemos la tasa de atrasos por dia segun la varable Fecha-I

In [31]:
datos_test['Fecha-I'] = datos_test['Fecha-I'].dt.normalize()

Fechas = datos_test.groupby(['Fecha-I']
                           #'OPERA',
                          #'MES',
                          #'DIA',
                          #,'temporada_alta'
                        , dropna=False, as_index= False).agg(
    Cantidad_vuelos = ('atraso_15','count'),
    Cantidad_atrasos = ('atraso_15','sum'))
                    
Fechas['Tasa de atrasos'] = Fechas['Cantidad_atrasos']/Fechas['Cantidad_vuelos']*100
Fechas.sort_values('Tasa de atrasos',ascending=False)



,Fecha-I,Cantidad_vuelos,Cantidad_atrasos,Tasa de atrasos
195,2017-07-15,159,99,62.264151
205,2017-07-25,177,109,61.581921
190,2017-07-10,198,115,58.080808
354,2017-12-21,225,110,48.888889
191,2017-07-11,195,93,47.692308
...,...,...,...,...
226,2017-08-15,183,11,6.010929
79,2017-03-21,167,10,5.988024
262,2017-09-20,189,11,5.820106
105,2017-04-16,181,8,4.419890


#### Observamos que los 3 valores de tasa de atrasos mas altos se presentan en el mes de Julio, veamos estos valores en un gráfico de serie temporal

In [32]:

fig = px.line(Fechas
              , x='Fecha-I'
              , y='Tasa de atrasos'
              , markers = True
              )
              

fig.show()

#### Calculo de tasa de atrasos por día de la semana

In [33]:
Dia = datos_test.groupby([
                          #'Des-O'
                          #'OPERA'
                          #'MES',
                          'DIANOM'
                          #'temporada_alta'
                        ],  dropna=False, as_index=False).agg(
    Cantidad_vuelos = ('atraso_15','count'),
    Cantidad_atrasos = ('atraso_15','sum'))
Dia['Tasa de atrasos'] = (Dia['Cantidad_atrasos']/Dia['Cantidad_vuelos'])*100
Dia.sort_values('Tasa de atrasos',ascending=False)

,DIANOM,Cantidad_vuelos,Cantidad_atrasos,Tasa de atrasos
6,Viernes,10292,2285,22.201710
2,Lunes,10131,2048,20.215181
1,Jueves,10251,2045,19.949273
4,Miercoles,9722,1661,17.084962
3,Martes,9662,1634,16.911613
5,Sabado,8352,1363,16.319444
0,Domingo,9796,1578,16.108616


In [34]:
fig = px.bar(Dia.sort_values('Tasa de atrasos',ascending=True)
             , y='DIANOM'
             , x='Tasa de atrasos'
             , color='Cantidad_vuelos'
             , color_continuous_scale= 'Bluered'
             , title= 'Tasa de atrasos y cantidad de vuelos según día de la semama'
             , text_auto=True)
             
fig.update_traces( marker_line_color = 'black')
fig.show()

### Observamos que los dias Viernes son los que presenetan mayor tasa de atrasos en cantidad de vuelos

#### Tasa de atrasos por Temporada

In [35]:
Temporada = datos_test.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'temporada_alta'
                        ], dropna=False, as_index=False).agg(
    Cantidad_vuelos = ('atraso_15','count'),
    Cantidad_atrasos = ('atraso_15','sum'))

Temporada['Tasa de atrasos'] = (Temporada['Cantidad_atrasos']/Temporada['Cantidad_vuelos'])*100

Temporada.sort_values('Tasa de atrasos',ascending=False)

def f(row):
    if row == 0:
        val = 'Temporada Baja'
    else:
        val = 'Temporada Alta'
    return val
Temporada['temporada_alta'] = Temporada['temporada_alta'].apply(f)
Temporada


,temporada_alta,Cantidad_vuelos,Cantidad_atrasos,Tasa de atrasos
0,Temporada Baja,45995,8254,17.945429
1,Temporada Alta,22211,4360,19.629913


In [36]:
fig = px.bar(Temporada
             , y ='temporada_alta'
             , x ='Tasa de atrasos'
             , color='Cantidad_atrasos'
             , color_continuous_scale= 'sunset'
             , orientation= 'h'
             , title = 'Tasa de atrasos por temporada coloreado por Cantidad de atrasos'
             , text_auto= True
             )

fig.update_traces(marker_line_color = 'black')
fig.show()

#### Observamos que para la temporada alta existe prácticamente la mitad de los vuelos con atrasos que en temporada alta, y las tasas de atrasos son muy parecidos, por lo que podemos decir que en temporada altas son mas eficientes en las salidas de vuelos

In [37]:
Feriados = datos_test.groupby([
                          #'Des-O'
                          #'OPERA',
                          #'MES',
                          #'DIA',
                          'Feriado'
                        ], dropna=False, as_index=False).agg(
   Cantidad_vuelos = ('atraso_15','count'),
   Cantidad_atrasos = ('atraso_15','sum')
                                            )

Feriados['Tasa de atrasos'] = (Feriados['Cantidad_atrasos']/Feriados['Cantidad_vuelos'])*100

Feriados = Feriados.sort_values('Tasa de atrasos',ascending=False)
Feriados


,Feriado,Cantidad_vuelos,Cantidad_atrasos,Tasa de atrasos
1,No Feriado,64973,12114,18.644668
0,Feriado,3233,500,15.465512


In [38]:
fig = px.bar(Feriados.sort_values('Tasa de atrasos'
             ,ascending=True)
             , y='Feriado'
             , x='Tasa de atrasos'
             , color= 'Cantidad_atrasos'
             , color_discrete_sequence=['steelblue']
             , text_auto= True)
fig.show()


#### Las tasas de atrasos en días feriados se presentan en 18.64% para los días no feriados vs 15.46% para los días feriados, y se observa una clara diferencia entre los atrasos de estos tipos de días.

# Conclusiones
### Observamos que entre las variables de periodos temporales, todas no indican que en el mes de Julio para el año 2017 se presentan una mayor tasa de atrasos en los vuelos, por otra parte el País que presenta mayor tasa en los atrasos de sus vuelos es Australia en conjunto con sus Aeropuertos. El dia que mayor tasa de atrasos tiene es el dia viernes dentro de todas las semanas. 
### Por otra parte las aerolineas que mas atrasos presentan en tasa son Plus Ultra Lineas Aereas, Qantas Airways y Air Canada 
### También vemos una leve diferencia entre los dias Feriados y no Feriados aumentando la tasa de atrasosen los dias no feriados.

# 4.-  Entrena uno o varios modelos (usando el/los algoritmo(s) que prefieras) para estimar la probabilidad de atraso de un vuelo. Siéntete libre de generar variables adicionales y/o complementar con variables externas

### MODELACIÓN


## Random Forest

#### Primero veremos la cantidad de datos que tenemos con atrasos y nos encontramos que  no se encuentra totalmente balanceado, por lo que crearemos dos modelos de random forest uno con los datos reales como vienen y otro con re muestreados.

In [39]:
print(datos_test['atraso_15'].value_counts())

0    55592
1    12614
Name: atraso_15, dtype: int64


In [40]:
#Resample
from sklearn.utils import resample


datos_no_atraso = datos_test[datos_test['atraso_15'] == 0]
datos_atraso = datos_test[datos_test['atraso_15'] == 1]

datos_atraso_sampled = resample(datos_atraso, 
                                 replace = True,     
                                 n_samples = 30000,    
                                 random_state = 666) 

data_sampled = pd.concat([datos_no_atraso, datos_atraso_sampled])

In [41]:
print(data_sampled['atraso_15'].value_counts())

0    55592
1    30000
Name: atraso_15, dtype: int64


#### en los datos re-muestreados dejaremos un n de 30000 registros con atrasos y la misma cantidad de datos no atrasados.

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report,roc_auc_score, roc_curve
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn import preprocessing

le = preprocessing.LabelEncoder()

rf = RandomForestClassifier()
rf1 = RandomForestClassifier()
#sfs = SequentialFeatureSelector(rf)

X = datos_test[[        'OPERA'
                        ,'periodo_dia'
                        ,'DIANOM'
                        ,'Mes_Nombre'
                        ,'temporada_alta'
                        ,'Destino'
                        #, 'hora_dia'
                        , 'hora'
                        , 'minutos'
                        ,'Feriado'
                        ,'TIPOVUELO'
                        ,'País']]
Y = datos_test[['atraso_15']]

### Las Variables Vlo-I, Ori-I, Vlo-O, Ori-O, SIGLAORI no se usaran en el modelo ya que presentan solo una categoría o tiene homologaciones de variables creadas como destino de operación y sus códigos.

In [43]:

X1 = data_sampled[[
                'OPERA'
                ,'periodo_dia'
                ,'DIANOM'
                ,'Mes_Nombre'
                ,'temporada_alta'
                ,'Destino'
                #, 'hora_dia'
                , 'hora'
                , 'minutos'
                ,'Feriado'
                ,'TIPOVUELO'
                ,'País']]
Y1 = data_sampled[['atraso_15']]


#### Pasaremos a crear label encoder para las variables del tipo categóricas, ya que los modelos de machine learning muchas veces no aceptan variables categóricas como objetos o string, así las pasamos a numéricas.

In [44]:
%%capture
X['OPERA'] = le.fit_transform(X['OPERA'])
X['DIANOM'] = le.fit_transform(X['DIANOM'])
X['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X['Mes_Nombre'] = le.fit_transform(X['Mes_Nombre'])
X['temporada_alta'] = le.fit_transform(X['temporada_alta'])
X['Destino'] = le.fit_transform(X['Destino'])
X['Feriado'] = le.fit_transform(X['Feriado'])
X['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X['País'] = le.fit_transform(X['País'])

In [45]:
%%capture
X1['OPERA'] = le.fit_transform(X1['OPERA'])
X1['DIANOM'] = le.fit_transform(X1['DIANOM'])
X1['periodo_dia'] = le.fit_transform(X1['periodo_dia'])
X1['Mes_Nombre'] = le.fit_transform(X1['Mes_Nombre'])
X1['temporada_alta'] = le.fit_transform(X1['temporada_alta'])
X1['Destino'] = le.fit_transform(X1['Destino'])
X1['Feriado'] = le.fit_transform(X1['Feriado'])
X1['TIPOVUELO'] = le.fit_transform(X1['TIPOVUELO'])
X1['País'] = le.fit_transform(X1['País'])

In [46]:

from sklearn.model_selection import train_test_split

# 80 % del conjunto de datos para entrenamiento y 20 % para validacion
X_train, X_test, Y_train, Y_test = train_test_split(
                                              X
                                            , Y
                                            ,test_size=0.2
                                            ,  random_state = 666)

# 80 % del conjunto de datos para entrenamiento y 20 % para validacion
X1_train, X1_test, Y1_train, Y1_test = train_test_split(
                                              X1
                                            , Y1
                                            ,test_size=0.2
                                            ,  random_state = 666)



In [47]:
rf.fit(X_train,Y_train)
rf1.fit(X1_train,Y1_train)


C:\Users\shado\AppData\Local\Temp\ipykernel_32352\2029393684.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

C:\Users\shado\AppData\Local\Temp\ipykernel_32352\2029393684.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier()

#### A medida de comparación de modelos con datos sin balancear y modelo con datos balanceados usaremos dos modelos uno con terminación "1" para mostrar que es con los datos balanceados.

In [71]:

Y_pred = rf.predict(X_test)

Y1_pred = rf1.predict(X1_test)


#### Obtenemos predicciones de ambos modelos

In [72]:
print("Reporte de clasificación de modelo desbalanceado")
print(classification_report(Y_test, Y_pred))
print("Reporte de clasificación de modelo balanceado")
print(classification_report(Y1_test, Y1_pred))

Reporte de clasificación de modelo desbalanceado
              precision    recall  f1-score   support

           0       0.79      0.96      0.87     11203
           1       0.87      0.53      0.66      5916

    accuracy                           0.81     17119
   macro avg       0.83      0.74      0.76     17119
weighted avg       0.82      0.81      0.80     17119

Reporte de clasificación de modelo balanceado
              precision    recall  f1-score   support

           0       0.86      0.83      0.84     11203
           1       0.69      0.74      0.72      5916

    accuracy                           0.80     17119
   macro avg       0.78      0.78      0.78     17119
weighted avg       0.80      0.80      0.80     17119



#### en el reporte de clasificación del modelo balanceado observamos mejoras en el recall de la categoría 1 que significa un aumento en las predicciones correctas de la categoría vuelo atrasado.

#### Generación métricas para Curva de ROC modelo desbalanceado

In [50]:
import plotly.express as px
from sklearn.metrics import roc_curve, auc
from sklearn.datasets import make_classification


y_score = rf.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(Y_test,y_score)

#### Curva ROC modelo desbalanceado

In [51]:
fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curva (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'), 
    width=700, height=700
)


fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1, 
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.update_traces(patch = {'line':{'color':'#ff2800', 'dash':'dash'}})
fig.show()

#### Generación metricas para Curva de ROC modelo balanceado

In [52]:
y1_score = rf1.predict_proba(X1_test)[:, 1]

fpr1, tpr1, thresholds = roc_curve(Y1_test,y1_score)

#### Curva ROC modelo balanceado

In [53]:
fig = px.area(
    x=fpr1, y=tpr1,
    title=f'ROC Curve (AUC={auc(fpr1, tpr1):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'), 
    width=700, height=700
)


fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1, 
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.update_traces(patch = {'line':{'color':'#ff2800', 'dash':'dash'}})
fig.show()

#### Observamos un aumento importante en lo que es la curva de ROC en la correcta clasificación de los atrasos en el modelo con balanceo.

#### Ahora pasaremos a generar un modelo Random Forest balanceado, mejorando algunos de sus hiper-parametros.

In [54]:
from sklearn.model_selection import GridSearchCV

max_features_range = np.arange(1,10,1)
n_estimators_range = np.arange(1,10,1)
param_grid = dict(max_features = max_features_range, n_estimators = n_estimators_range)

grid = GridSearchCV(estimator = rf1, param_grid = param_grid, cv=10)


In [73]:
%%capture
grid.fit(X1_train, Y1_train)


In [74]:
print("Mejor combinacion de hiper-parametros para modelo con balanceo es  %s con precición de  %0.4f"
      % (grid.best_params_, grid.best_score_))

Mejor combinacion de hiper-parametros para modelo con balanceo es  {'max_features': 1, 'n_estimators': 8} con precición de  0.7861


#### Nuestro nuevo modelo se llamara rf_final y se generara desde la muestra balanceada y con los hiper-parametros optimizados según la grilla grid.

In [75]:
rf_final = RandomForestClassifier(   max_features = grid.best_params_['max_features']
                                   , n_estimators = grid.best_params_['n_estimators']
                                    )
rf_final.fit(X1_train,Y1_train)

Y1_pred = rf_final.predict(X1_test)

C:\Users\shado\AppData\Local\Temp\ipykernel_32352\1071643404.py:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [76]:
print("Reporte de clasificación de modelo balanceado y optimizado")

print(classification_report(Y1_test,Y1_pred))

y_score = rf_final.predict_proba(X_test)[:, 1]

fpr_final, tpr_final, thresholds = roc_curve(Y_test,y_score)



Reporte de clasificación de modelo balanceado y optimizado
              precision    recall  f1-score   support

           0       0.85      0.83      0.84     11203
           1       0.69      0.73      0.71      5916

    accuracy                           0.79     17119
   macro avg       0.77      0.78      0.77     17119
weighted avg       0.80      0.79      0.79     17119



#### Obtenemos ligeros cambios en las estadísticas de reporte, lo que es de esperarse en las optimizaciones de hiper-parametros

In [59]:
fig = px.area(
    x=fpr1, y=tpr1,
    title=f'ROC Curve (AUC={auc(fpr_final, tpr_final):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'), 
    width=700, height=700
)


fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1, 
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.update_traces(patch = {'line':{'color':'#ff2800', 'dash':'dash'}})
fig.show()

#### Observamos una mejora en el porcentaje de clasificación correcta que presenta la curva de ROC, por lo que la busqueda y optimización de hiper-parametros funciono correctamente

#### Ahora se probaran 2 modelos mas, un Gradient boosted y un árbol de decisión, según la misma lógica anterior.
#### Se usará la muestra balanceada y si estos modelos mejoran los ajustes según sus métricas se harán las observaciones para elegir el mejor modelo de los 3 tipos.


### Gradient Boosted

In [60]:
%%capture
import xgboost as xgb

modelo_xgb = xgb.XGBClassifier()

X = data_sampled[[        'OPERA'
                        ,'periodo_dia'
                        ,'DIANOM'
                        ,'Mes_Nombre'
                        ,'temporada_alta'
                        ,'Destino'
                        #, 'hora_dia'
                        , 'hora'
                        , 'minutos'
                        ,'Feriado'
                        ,'TIPOVUELO'
                        ,'País']]
Y = data_sampled[['atraso_15']]


In [61]:
%%capture
X['OPERA'] = le.fit_transform(X['OPERA'])
X['DIANOM'] = le.fit_transform(X['DIANOM'])
X['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X['Mes_Nombre'] = le.fit_transform(X['Mes_Nombre'])
X['temporada_alta'] = le.fit_transform(X['temporada_alta'])
X['Destino'] = le.fit_transform(X['Destino'])
X['Feriado'] = le.fit_transform(X['Feriado'])
X['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X['País'] = le.fit_transform(X['País'])

In [62]:

X_train, X_test, Y_train, Y_test = train_test_split( X
                                                    ,Y
                                                    ,test_size=0.2
                                                    , random_state= 666)

In [63]:
%%capture
modelo_xgb.fit(X_train,Y_train)

In [78]:

Y_pred = modelo_xgb.predict(X_test)

y_score = modelo_xgb.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(Y_test,y_score)


print("Reporte de clasificación de modelo Gradient boosted balanceado")
print(classification_report(Y_test,Y_pred))


Reporte de clasificación de modelo Gradient boosted balanceado
              precision    recall  f1-score   support

           0       0.75      0.88      0.81     11203
           1       0.67      0.45      0.54      5916

    accuracy                           0.73     17119
   macro avg       0.71      0.66      0.67     17119
weighted avg       0.72      0.73      0.72     17119



In [79]:
fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curva (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'), 
    width=700, height=700
)


fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1, 
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.update_traces(patch = {'line':{'color':'#50339E', 'dash':'dash'}})
fig.show()


### Árbol 

In [66]:
from sklearn.tree import DecisionTreeClassifier
modelo_arbol = DecisionTreeClassifier()

X = data_sampled[[       'OPERA'
                        ,'periodo_dia'
                        ,'DIANOM'
                        ,'Mes_Nombre'
                        ,'temporada_alta'
                        ,'Destino'
                        #, 'hora_dia'
                        , 'hora'
                        , 'minutos'
                        ,'Feriado'
                        ,'TIPOVUELO'
                        ,'País']]
Y = data_sampled[['atraso_15']]

In [67]:
%%capture
X['OPERA'] = le.fit_transform(X['OPERA'])
X['DIANOM'] = le.fit_transform(X['DIANOM'])
X['periodo_dia'] = le.fit_transform(X['periodo_dia'])
X['Mes_Nombre'] = le.fit_transform(X['Mes_Nombre'])
X['temporada_alta'] = le.fit_transform(X['temporada_alta'])
X['Destino'] = le.fit_transform(X['Destino'])
X['Feriado'] = le.fit_transform(X['Feriado'])
X['TIPOVUELO'] = le.fit_transform(X['TIPOVUELO'])
X['País'] = le.fit_transform(X['País'])

In [68]:

X_train, X_test, Y_train, Y_test = train_test_split( X
                                                    ,Y
                                                    ,test_size=0.2
                                                    , random_state= 666)

In [80]:
modelo_arbol.fit(X_train,Y_train)
Y_pred = modelo_arbol.predict(X_test)

y_score = modelo_arbol.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(Y_test,y_score)

print("Reporte de clasificación de modelo árbol de decisión balanceado")
print(classification_report(Y_test,Y_pred))

Reporte de clasificación de modelo árbol de decisión balanceado
              precision    recall  f1-score   support

           0       0.84      0.82      0.83     11203
           1       0.67      0.70      0.68      5916

    accuracy                           0.77     17119
   macro avg       0.75      0.76      0.75     17119
weighted avg       0.78      0.77      0.78     17119



In [81]:

fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curva (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'), 
    width=700, height=700
)


fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1, 
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.update_traces(patch = {'line':{'color':'#102CC4', 'dash':'dash'}})
fig.show()


# 5.-  Evalúa tu modelo. ¿Qué performance tiene? ¿Qué métricas usaste para evaluar esa performance y por qué? ¿Por qué elegiste ese algoritmo en particular? ¿Qué variables son las que más influyen en la predicción? ¿Cómo podrías mejorar la performance?


# Conclusiones:
#### De los 3 tipos de modelos entrenados en el problema, Random Forest, Gradient boosted y arboles de decisión nos quedaremos como mejor modelo el Random Forest ya que al generarlo balanceado este  mejora sustantivamente las métricas para las correctas clasificaciones de los atrasos en los vuelos. y mejoramos las estadísticas de performance de este modelo con una búsqueda intencionada de parámetros en una grilla.

### Mejor modelo es Random Forest con una muestra balanceada con  'max_features = 1 y  'n_estimators' = 8


# Agradecimientos:
#### esta sección quiero presentar mi gratitud en la oportunidad de resolver el desafío, lo pase muy bien y me entretuve bastante. Saludos.
